# **Coordinate to impulse space Neural Network** 

In this notebook I will attempt to create Neural Network that learns transformation from coordinate to impulse space.

Theoretically it should work well for particles that were born in origin.

Here I prepare data for the network.

Data preparation
---
* remove particle_id == 0
* remove negative charge particles
* remove non origin particles

In [1]:
# Imports

import os

import numpy as np
import pandas as pd

In [2]:
# Load data

data_path = '../Data/train_100_events/'
event_prefix = 'event000001000'
hits, cells, particles, truth = load_event(os.path.join(data_path, event_prefix))

In [3]:
# Shuffle hits
from sklearn.utils import shuffle
hits = shuffle(hits)

In [5]:
# Prepare data

x_train = [] # np.transpose([hits.x.values, hits.y.values, hits.z.values])
y_train = []
progress = 0

for index, hit in hits.iterrows():
    hit_id = hit['hit_id']
    truth_of_hit = truth.loc[truth.hit_id == hit_id]
    particle_id = truth_of_hit['particle_id'].values[0]    
    if particle_id == 0: continue
        
    particle_info = particles[particles.particle_id == particle_id]
    charge = particle_info['q'].values[0]    
    if charge > 0: continue
        
    origin = [particle_info['vx'].values[0],
              particle_info['vy'].values[0]]
    
    if np.linalg.norm(origin) > 0.1: continue
        
    coordinates = [hit['x'], hit['y'], hit['z']]
    impulse = [particle_info['px'].values[0],
               particle_info['py'].values[0],
               particle_info['pz'].values[0]]
    
    x_train.append(coordinates)
    y_train.append(impulse)
    
    if len(x_train)%1000 == 0 : 
        progress+=1
        print(progress)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [6]:
# Inspect data
np.set_printoptions(precision=5, suppress=True)
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
print(x_train[0:10])
print(y_train[0:10])

[[  -93.9573   -652.93701 -1067.27002]
 [ -553.203      39.2662  -1215.5    ]
 [ -866.11798  -209.32201 -1502.5    ]
 [  -27.839     -20.9165    602.5    ]
 [ -233.84399   119.106    -341.20001]
 [ -156.27299  -209.965     328.79999]
 [ -203.909     161.52      540.40002]
 [  591.22699   665.70398 -2952.5    ]
 [ -279.00201   597.13397   522.20001]
 [  -31.4884    -12.6849    131.81599]]
[[-0.22612 -0.17179 -0.41093]
 [-0.14429  0.1768  -0.42881]
 [-0.43994  0.16702 -0.74537]
 [-0.18919 -0.12845  3.97079]
 [-0.3989   0.29185 -0.63017]
 [-0.77344 -0.91488  1.50366]
 [-0.49851  0.49538  1.46384]
 [ 1.13041  0.93099 -4.86151]
 [-0.35318  1.22197  1.01961]
 [-0.17881 -0.06086  0.79745]]


In [7]:
# Store data
save_path = '../Data/generated_data/'
x_train_df = pd.DataFrame(x_train)
y_train_df = pd.DataFrame(y_train)
x_train_df.to_csv(os.path.join(data_path, "Coordinates_to_impulse_X.csv"))
y_train_df.to_csv(os.path.join(data_path, "Coordinates_to_impulse_Y.csv"))